In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
import os
#from dotenv import load_dotenv

#load_dotenv(verbose=True)
ES_CLOUD_ID = 'f950ade65a814f45b5b8e877d966S1527:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvJGJmYzNmZDY4MjdiOTRhNGVhOGQ1NmY5ODA1YTQ0Y2FlJGNhYzk3NmUyZTMyMzQ0YjJhNTU3MTdjZTk4OTM5Mjhl'
ES_USER = '4177922518'
ES_PASSWORD = 'ghdtjrwn1!'
ES_API_KEY = 'RTVnOUFwRUJHY25xVFJJQWh6bVo6c0JYZm1keGRRblNaU25IM0pQdE56QQ=='
index_name = "helloworld7"

In [2]:
import torch

torch.cuda.empty_cache()

In [3]:
import json

CONFIG_NAME = "config.json"
with open(f'configs/{CONFIG_NAME}', 'r') as f:
    config = json.load(f)

In [4]:
# # Embeddings by llama.cpp quantized model
# # llama-cpp-python 설치 필요
# # from transformers import AutoTokenizer, AutoModelForCausalLM
# # from langchain_community.llms import LlamaCpp
# from typing import List
# from langchain_community.embeddings import LlamaCppEmbeddings

# class BllossomEmbeddings:
#         def __init__(self):
#             self.model = LlamaCppEmbeddings(
#                 model_path=config['config']['quantized_path'],
#                 n_gpu_layers=-1,
#                 n_batch=config['config']['n_batch'],
#                 # callback_manager=callback_manager,
#                 # temperature=config['inference']['temperature'],
#                 # top_p=config['inference']['top_p'],
#                 # top_k=config['inference']['top_k'],
#                 # max_tokens=config['inference']['max_length'],
#                 verbose=True,  # Verbose is required to pass to the callback manager
#             )

#         # AttributeError: 'LlamaCpp' object has no attribute 'encode'
#         def embed_documents(self, texts: List[str]) -> List[List[float]]:
#             return [self.model.encode(t).tolist() for t in texts]
        
#         def embed_query(self, query: str) -> List[float]:
#             return self.model.encode([query])


#         # def embed_documents(self, texts: List[str]) -> List[List[float]]:
#         #     return [self._get_embeddings(t) for t in texts]
        
#         # def embed_query(self, query: str) -> List[float]:
#         #     return self._get_embeddings(query)
        
#         # def _get_embeddings(self, text: str) -> List[float]:
#         #     output = self.model(text)
#         #     print('## output : ', output)
#         #     # Assuming output is a list of token embeddings, we average them to get a fixed-size embedding
#         #     # embeddings = np.mean(output, axis=0)
#         #     return output

In [5]:
# from langchain.embeddings import HuggingFaceEmbeddings
from typing import List
from langchain_community.embeddings import LlamaCppEmbeddings

def setup_embeddings():
    # Huggingface embedding setup
    # print(">> Prep. Huggingface embedding setup")
    print(">> Prep. Llama-cpp Quantized embedding setup")
    embeddings = LlamaCppEmbeddings(
                model_path=config['config']['quantized_path'],
                n_gpu_layers=-1,
                n_batch=config['config']['n_batch'],
                n_ctx=config['config']['n_ctx'],
                # callback_manager=callback_manager,
                # temperature=config['inference']['temperature'],
                # top_p=config['inference']['top_p'],
                # top_k=config['inference']['top_k'],
                # max_tokens=config['inference']['max_length'],
    )
    return embeddings
    # return HuggingFaceEmbeddings(model_name=model_name, cache_folder="./cache", model_kwargs={"device" : "cuda:0"})

hf = setup_embeddings()

>> Prep. Llama-cpp Quantized embedding setup


llama_model_loader: loaded meta data with 32 key-value pairs and 291 tensors from bllossom/llama-3-Korean-Bllossom-8B-Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Elo_Best_Chat_Model
llama_model_loader: - kv   3:                       general.organization str              = MLP KTLim
llama_model_loader: - kv   4:                         general.size_label str              = 8.0B
llama_model_loader: - kv   5:                            general.license str              = llama3
llama_model_loader: - kv   6:                   general.base_model.count u32              = 1
llama_model_loader: - kv   7:                

In [6]:
# ElasticSearch vectorstore in langchain style
from langchain_elasticsearch import ElasticsearchStore

db = ElasticsearchStore(
    es_cloud_id=ES_CLOUD_ID,
    es_user=ES_USER,
    es_password=ES_PASSWORD,
    es_api_key=ES_API_KEY,
    index_name=index_name,
    embedding=hf,
    #es_url = 'https://bfc3fd6827b94a4ea8d56f9805a44cae.us-central1.gcp.cloud.es.io:443'
)

In [7]:
index_name

'helloworld7'

### Data chunking

In [8]:
import pandas as pd
from tqdm import tqdm

# 임시로 경로 변경
data = pd.read_csv('data/preprocessed.csv')
#data['text'] = ''
data.head()

,source,title,content
0,경기도외국인지원센터_상담사례,중도 퇴사 후 근로소득 신고되지 않아 고용허가연장 안된 노동자 지원,작성일: 23-11-27\n상담유형: 체류자격\n거주지역: 여주시\n국적: 우즈베키...
1,경기도외국인지원센터_상담사례,국민연금 반환일시금 신청 지원,작성일: 23-11-27\n상담유형: 기타\n거주지역: 남양주시\n국적: 필리핀\n...
2,경기도외국인지원센터_상담사례,외국인아동 교육권 보장을 위한 체류자격 부여자(G-1-85) 외국인등록 절차 지원,작성일: 23-11-27\n상담유형: 체류자격\n거주지역: 남양주시\n국적: 필리핀...
3,경기도외국인지원센터_상담사례,단체행동을 통해 체불임금 해결한 사례,작성일: 23-11-27\n상담유형: 임금체불\n거주지역: 안산시\n국적: 필리핀\...
4,경기도외국인지원센터_상담사례,취업가능기간이 짧게 남아 취업이 어려운 노동자 구직활동 지원,작성일: 23-11-27\n상담유형: 고용허가\n거주지역: 시흥시\n국적: 필리핀\...


In [9]:
data.shape

(3433, 3)

In [10]:
# from langchain.text_splitter import RecursiveCharacterTextSplitter

# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size = 500,
#     chunk_overlap  = 100,
#     length_function = len,
# )

# #texts = text_splitter.split_text(data[0].page_content)
# texts = text_splitter.split_text(data['content'][0])
# len(texts)


In [11]:
from transformers import AutoTokenizer

# LLaMA-3 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("MLP-KTLim/llama-3-Korean-Bllossom-8B")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:99: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
from langchain.docstore.document import Document
# import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

# RAG style : title + [SEP] + contents
# seperation 지정되어 있지 않음 -> 공백으로 처리
def make_chunk_data(titles: list, contents: list) -> list:
    if tokenizer.sep_token != None:
        print('## seperated by : ', tokenizer.sep_token)
        chunk_data = [title + tokenizer.sep_token + content for title, content in zip(titles, contents)]
    else:
        chunk_data = [title + " " + content for title, content in zip(titles, contents)]

    return chunk_data


# 토크나이저 기준 분할
def data_chunking(titles: list, contents: list, sources: list) -> Document:
    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        tokenizer,
        chunk_size=config['config']['chunk_size'],
        chunk_overlap=config['config']['overlap_size']
    )
    chunk_data = make_chunk_data(titles, contents)
    chunks = text_splitter.create_documents(chunk_data, sources)

    return chunks

In [13]:
data.head()

,source,title,content
0,경기도외국인지원센터_상담사례,중도 퇴사 후 근로소득 신고되지 않아 고용허가연장 안된 노동자 지원,작성일: 23-11-27\n상담유형: 체류자격\n거주지역: 여주시\n국적: 우즈베키...
1,경기도외국인지원센터_상담사례,국민연금 반환일시금 신청 지원,작성일: 23-11-27\n상담유형: 기타\n거주지역: 남양주시\n국적: 필리핀\n...
2,경기도외국인지원센터_상담사례,외국인아동 교육권 보장을 위한 체류자격 부여자(G-1-85) 외국인등록 절차 지원,작성일: 23-11-27\n상담유형: 체류자격\n거주지역: 남양주시\n국적: 필리핀...
3,경기도외국인지원센터_상담사례,단체행동을 통해 체불임금 해결한 사례,작성일: 23-11-27\n상담유형: 임금체불\n거주지역: 안산시\n국적: 필리핀\...
4,경기도외국인지원센터_상담사례,취업가능기간이 짧게 남아 취업이 어려운 노동자 구직활동 지원,작성일: 23-11-27\n상담유형: 고용허가\n거주지역: 시흥시\n국적: 필리핀\...


In [14]:
from tqdm import tqdm
# 빈 리스트 초기화
titles = []
contents = []
sources = []

# 데이터프레임 순회
for index, row in data.iterrows():
    titles.append(row['title'])
    contents.append(row['content'])
    sources.append({'source': row['source']})

# 결과 확인
#print("Titles:", titles)
#print("Contents:", contents)

In [15]:
len(titles), len(contents), len(sources)

(3433, 3433, 3433)

In [16]:
chunks = data_chunking(titles, contents, sources)

In [17]:
len(chunks)

8360

In [18]:
chunks[0:2]

[Document(metadata={'source': '경기도외국인지원센터_상담사례'}, page_content='중도 퇴사 후 근로소득 신고되지 않아 고용허가연장 안된 노동자 지원 작성일: 23-11-27\n상담유형: 체류자격\n거주지역: 여주시\n국적: 우즈베키스탄\n체류자격: E-9\n상담내용: 중도 퇴직하는 외국인 근로자의 추가 납부세액 사례\n진행 과정 및 결과: 2023-09-18\n- 고용노동부에서 외국인근로자의 2020년 종합소득세 체납을   확인하여 고용허가제 연장 불가능 안내받았음 \n 회사 담당자하고 여주세무서에 방문 및 체납된 세금의 원인 상담 내용을 통역 요청으로 전화 상담 \n타행송금; 자동차세-7.360원, 2021년 주민세-11,330원, 2022년 주민세-11,330원, 2022년 지방세 44,630원, 2022년 소득세 – 476,590원, 2021년 지방세 173,140원, \n공과금 2020년 1,808,950원 안내 통역함\n\n2020-09-20\n- 2021년에 연말정산 신청하여 469,520원 환급받았음 \n- 체류자격 연장을 위해서 합계 2,533,330원을 납부함 \n공과금 납부 내용을 상담하러 이천세무서에 동시 통역 가능한지 문의로 전화 상담 \n10월 21일 오전 10시에 이천세무서에 방문 예정 안내함'),
 Document(metadata={'source': '경기도외국인지원센터_상담사례'}, page_content='2023-09-21\n- 이천세무서 3층 소득세과에 방문 상담 내용 통번역 함\n- 1,808,000원 2020년도의 체납 안내: \n외국인근로자의 2020년에 중도에 퇴직후에 근로소득 신고하지 않아서 추가 체납 확인 \n* 근로자가 중도에 퇴직하는 경우 원천징수의무자(회사)는 퇴직하는 달의 근로소득을 지급하는 때에 근로소득 세액을 징수함.\n * 따라서, 중도 퇴직하는 근로자의 경우 퇴직하는 달의 급여를 받기 전 회사에 근로소득자의 소득·세액공제신고서와 증빙서류를 제출하여야 함  \n* 관련 

In [19]:
index_name

'helloworld7'

In [20]:
# """
# 런타임 모두 실행 시 DB에 데이터가 중복 저장되는 것을 막기 위해 주석 처리 해놨습니다.
# 필요한 코드이니 지우지 말아주세요.
# """
#DB에 텍스트 데이터 추가

#꼭 확인!!
index_name = index_name

db.from_documents(chunks,
              embedding=hf,
              es_cloud_id=ES_CLOUD_ID,
              es_user=ES_USER,
              es_password=ES_PASSWORD,
              es_api_key=ES_API_KEY,
              index_name=index_name)


llama_print_timings:        load time =   26788.38 ms
llama_print_timings:      sample time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time =   26748.72 ms /   256 tokens (  104.49 ms per token,     9.57 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =   26825.13 ms /   257 tokens

llama_print_timings:        load time =   26788.38 ms
llama_print_timings:      sample time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time =   26437.89 ms /   256 tokens (  103.27 ms per token,     9.68 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =   26531.85 ms /   257 tokens

llama_print_timings:     